In [ ]:
import collections
import operator

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from pyclustering.container.kdtree import kdtree_balanced, kdtree_visualizer
from pyclustering.utils import read_sample
from pyclustering.samples.definitions import FCPS_SAMPLES

In [ ]:
from shapely.geometry import Point, Polygon

In [ ]:
import scipy as sp

## Implementation by hand

source:
    https://johnlekberg.com/blog/2020-04-17-kd-tree.html

In [ ]:
def SED(X, Y):
    """Compute the squared Euclidean distance between X and Y."""
    return sum((i-j)**2 for i, j in zip(X, Y))

In [ ]:
SED( (3, 4), (4, 9) )

#### brute force

In [ ]:
def nearest_neighbor_bf(*, query_points, reference_points):
    """Use a brute force algorithm to solve the
    "Nearest Neighbor Problem".
    """
    return {
        query_p: min(
            reference_points,
            key=lambda X: SED(X, query_p),
        )
        for query_p in query_points
    }

In [ ]:
reference_points = [ (1, 2), (3, 2), (4, 1), (3, 5) ]
query_points = [
    (3, 4), (5, 1), (7, 3), (8, 9), (10, 1), (3, 3)]

In [ ]:
nearest_neighbor_bf(reference_points = reference_points, query_points = query_points,)

### k-d trees algorithm

In [ ]:
BT = collections.namedtuple("BT", ["value", "left", "right"])
BT.__doc__ = """
A Binary Tree (BT) with a node value, and left- and
right-subtrees.
"""

In [ ]:
def kdtree(points):
    """Construct a k-d tree from an iterable of points.
    
    This algorithm is taken from Wikipedia. For more details,
    
    > https://en.wikipedia.org/wiki/K-d_tree#Construction
    
    """
    k = len(points[0])
    
    def build(*, points, depth):
        """Build a k-d tree from a set of points at a given
        depth.
        """
        if len(points) == 0:
            return None
        
        points.sort(key=operator.itemgetter(depth % k))
        middle = len(points) // 2
        
        return BT(
            value = points[middle],
            left = build(
                points=points[:middle],
                depth=depth+1,
            ),
            right = build(
                points=points[middle+1:],
                depth=depth+1,
            ),
        )
    
    return build(points=list(points), depth=0)

In [ ]:
reference_points = [ (1, 2), (3, 2), (4, 1), (3, 5) ]
kdtree(reference_points)

BT(value=(3, 5), 
left=BT(
   value=(3, 2), 
   left=BT(
      value=(1, 2), 
      left=None, 
      right=None), 
   right=None), 
right=BT(
   value=(4, 1), 
   left=None, 
   right=None))

In [ ]:
NNRecord = collections.namedtuple("NNRecord", ["point", "distance"])
NNRecord.__doc__ = """
Used to keep track of the current best guess during a nearest
neighbor search.
"""

In [ ]:
def find_nearest_neighbor(*, tree, point):
    """Find the nearest neighbor in a k-d tree for a given
    point.
    """
    k = len(point)
    
    best = None
    def search(*, tree, depth):
        """Recursively search through the k-d tree to find the
        nearest neighbor.
        """
        nonlocal best
        
        if tree is None:
            return
        
        distance = SED(tree.value, point)
        if best is None or distance < best.distance:
            best = NNRecord(point=tree.value, distance=distance)
        
        axis = depth % k
        diff = point[axis] - tree.value[axis]
        if diff <= 0:
            close, away = tree.left, tree.right
        else:
            close, away = tree.right, tree.left
        
        search(tree=close, depth=depth+1)
        if diff**2 < best.distance:
            search(tree=away, depth=depth+1)
    
    search(tree=tree, depth=0)
    return best.point

In [ ]:
reference_points = [ (1, 2), (3, 2), (4, 1), (3, 5) ]
tree = kdtree(reference_points)
find_nearest_neighbor(tree=tree, point=(10, 1))

### visualize the points

In [ ]:
plt.style.use('_mpl-gallery')

In [ ]:
# plot
fig, ax = plt.subplots()
x = [x[0] for x in reference_points] + [10]
y = [y[1] for y in reference_points] + [1]

# size and color:
sizes = np.random.uniform(15, 80, len(x))
colors = np.random.uniform(15, 80, len(x))

ax.scatter(x, y, s=sizes, c=colors, vmin=0, vmax=100)

ax.set(xlim=(0, 12), xticks=np.arange(1, 12),
       ylim=(0, 12), yticks=np.arange(1, 12))

plt.show()

## using pyclustering library to visualize kd tree geometry:
https://pyclustering.github.io/docs/0.10.1/html/d3/d38/classpyclustering_1_1container_1_1kdtree_1_1kdtree__visualizer.html

In [ ]:
p2 = [ (1, 2), (3, 2), (4, 1), (3, 5) ]

In [ ]:
tree_instance = kdtree_balanced(p2)
 
kdtree_visualizer(tree_instance).visualize()

#### using more points

generating random points within a bounding box:
https://www.matecdev.com/posts/random-points-in-polygon.html

In [ ]:
def Random_Points_in_Polygon(polygon, number):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < number:
        pnt = Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy))
        if polygon.contains(pnt):
            points.append(pnt)
    return points

In [ ]:
polygon = Polygon([[42.497830,-80.599547],
                   [43.362853, -80.423271],[43.5121265, -80.480605],[43.344230, -80.597123],
                   [42.497830,-80.599547]])
points = Random_Points_in_Polygon(polygon, 100)

# Plot the polygon
xp,yp = polygon.exterior.xy
plt.plot(xp,yp)

# Plot the list of points
xs = [point.x for point in points]
ys = [point.y for point in points]
plt.scatter(xs, ys,color="red")
plt.show()

In [ ]:
p3 = [(p.x,p.y) for p in points]

In [46]:
p3[0:10]

[(43.45526362184373, -80.49585778506294),
 (43.33160240688523, -80.58045570653258),
 (43.22779172243659, -80.5655706449469),
 (42.79058321229976, -80.5968121036445),
 (43.0158273856703, -80.594264265468),
 (43.35727724903474, -80.47852873424732),
 (42.86925341933145, -80.55109426467374),
 (42.73804493549406, -80.58457158549932),
 (43.27952280469559, -80.54655756744943),
 (43.01114177781527, -80.53465500276002)]

In [ ]:
tree_instance = kdtree_balanced(p3)
 
kdtree_visualizer(tree_instance).visualize()

## scipy spatial.kdtree implementation to create a tree and query it

In [ ]:
test_point = p3[50]
test_point

In [48]:
sp_tree = sp.spatial.KDTree(p3, balanced_tree=True)

In [49]:
sp_tree.size # number of nodes in the tree

31

In [52]:
sp_tree.n # the number of data points

100

In [54]:
sp_tree.data[50] # coordinate at index 50 of the set of points in the dataset

array([ 43.20050164, -80.5425378 ])

In [53]:
# the indexes of the points that are 0.025 from a given data point ([ 43.20050164, -80.5425378 ]))
sp_tree.query_ball_point([p3[50]], r=.025, return_sorted=True)

array([list([50, 69, 81])], dtype=object)

you can count the number of neighbours by using the count_neighbors method
https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.count_neighbors.html#scipy.spatial.KDTree.count_neighbors

In [ ]:
tree2 = sp.spatial.KDTree([p3[50]])
sp_tree.count_neighbors(tree2, r=0.025)

find the distances and the indexes of the k nearest neighbours of a point
https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.query.html#scipy.spatial.KDTree.query

In [ ]:
sp_tree.query([p3[50]], k=3)

In [ ]:
dist_array, data_index = sp_tree.query([p3[50]], k=3)

In [ ]:
nearest_points = [sp_tree.data[i] for i in data_index]

In [ ]:
nearest_points # to [ 43.20050164, -80.5425378 ]

how would you sort the array of points into n sized groupings